<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#AB-Testing" data-toc-modified-id="AB-Testing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>AB-Testing</a></span></li></ul></div>

# AB-Testing

![cats](images/cats.jpeg)


Imaginad que somos los cientificos de datos de la empresa de videojuegos Tactile Entertainment. Los desarrolladores del juego Cookie Cats pretenden introducir un cambio en el juego para aumentar la retencion de los jugadores. En cierto nivel del juego los jugadores se encuentran una puerta que les obliga a esperar o a pagar la app. Actualmente la puerta se encuentra en nivel 30 y se pretende pasar al nivel 40, para comprobar la retencion a 1 y 7 dias. Antes de realizar el cambio definitivo en el juego se raliza un test AB.

Los datos estan alojados en `data/cookie_cats.csv`. Nuestro grupo de control sera la version actual `gate_30` y el grupo de tratamiento sera la version `gate_40`. Debemos realizar el test para 1 dia de retencion `retention_1` y para 7 dias `retention_7`.

In [3]:
# librerias

import pandas as pd
import numpy as np

from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from scipy.stats import norm, sem

import pylab as plt
import seaborn as sns

In [4]:
# datos
TE_level_trail = pd.read_csv('data/cookie_cats.csv')


In [5]:
TE_level_trail.head(3)

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False


In [6]:
TE_level_trail.describe().T

,count,mean,std,min,25%,50%,75%,max
userid,90189.0,4.998412e+06,2.883286e+06,116.0,2512230.0,4995815.0,7496452.0,9999861.0
sum_gamerounds,90189.0,5.187246e+01,1.950509e+02,0.0,5.0,16.0,51.0,49854.0


In [7]:
TE_level_trail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [8]:
TE_level_trail.loc[(TE_level_trail.version=='gate_30'),['version','retention_1']].value_counts()

version  retention_1
gate_30  False          24666
         True           20034
dtype: int64

In [9]:
TE_level_trail.loc[(TE_level_trail.version=='gate_40'),['version','retention_7']]

,version,retention_7
2,gate_40,False
3,gate_40,False
4,gate_40,True
5,gate_40,True
7,gate_40,False
...,...,...
90181,gate_40,False
90184,gate_40,False
90185,gate_40,False
90187,gate_40,False


In [10]:
new_Rtrail = TE_level_trail.loc[(TE_level_trail.version=='gate_40'),['version','retention_7']]

In [11]:
old_Rtrail = TE_level_trail.loc[(TE_level_trail.version=='gate_30'),['version','retention_1']]

In [12]:
new_Rtrail.describe().value_counts

<bound method DataFrame.value_counts of         version retention_7
count     45489       45489
unique        1           2
top     gate_40       False
freq      45489       37210>

In [13]:
old_Rtrail.describe().T

,count,unique,top,freq
version,44700,1,gate_30,44700
retention_1,44700,2,False,24666


In [14]:
pd.crosstab(TE_level_trail.version,TE_level_trail.retention_1)

retention_1,False,True
version,,
gate_30,24666,20034
gate_40,25370,20119


In [15]:
pd.crosstab(TE_level_trail.version,TE_level_trail.retention_7)

retention_7,False,True
version,,
gate_30,36198,8502
gate_40,37210,8279


**Grupo_1**

**G1_Step_1**

In [23]:
control_res_1=TE_level_trail[TE_level_trail.version=='gate_30']['retention_1']
tratamiento_res_1=TE_level_trail[TE_level_trail.version=='gate_40']['retention_1']

**G1_Step_2** 

In [24]:
n_con_1=control_res_1.count()   # cuenta de elementos por grupo, ie, conversiones
n_trat_1=tratamiento_res_1.count()
n_con_1,n_trat_1

(44700, 45489)

**G1_Step_3**

In [25]:
conversiones_1=[control_res_1.sum(), tratamiento_res_1.sum()]  # conversiones por grupo
conversiones_1

[20034, 20119]

**G1_Step_4**

In [26]:
observaciones_1=[n_con_1, n_trat_1] # cuenta N de cada grupo
observaciones_1

[44700, 45489]

**G1_Step_5**

In [27]:
z_score_1, p_value_1 = proportions_ztest(conversiones_1, nobs=observaciones_1)
z_score_1, p_value_1

(1.7840862247974725, 0.07440965529691913)

**Grupo_2**

**G2_Step_1**

In [28]:
control_res_2=TE_level_trail[TE_level_trail.version=='gate_30']['retention_7']
tratamiento_res_2=TE_level_trail[TE_level_trail.version=='gate_40']['retention_7']

**G2_Step_2** 

In [31]:
n_con_2=control_res_2.count()   # cuenta de elementos por grupo, ie, conversiones
n_trat_2=tratamiento_res_2.count()

n_con_2,n_trat_2

(44700, 45489)

**G2_Step_3**

In [32]:
conversiones_2=[control_res_2.sum(), tratamiento_res_2.sum()]  # conversiones por grupo
conversiones_2

[8502, 8279]

**G2_Step_4**

In [33]:
observaciones_2=[n_con_2, n_trat_2] # cuenta N de cada grupo
observaciones_2

[44700, 45489]

**G2_Step_5**

In [34]:
z_score_2, p_value_2 = proportions_ztest(conversiones_2, nobs=observaciones_2)
z_score_2, p_value_2

(3.164358912748191, 0.001554249975614329)

**Conclusión:**

In [37]:
f'El grupo 1 que estimaba la retencion de 1 dia ha presentado un resultado de {round(p_value_1,5)} > 0.05, superior al umbal, por otro lado El grupo 2 que estimaba la retencion de un 7 dias ha presentado un resultado de {round(p_value_2,5)} < 0.05. Los resultados nos indican que el grupos de retencion a los 7 dias, obtendra mayor retencion'

'El grupo 1 que estimaba la retencion de 1 dia ha presentado un resultado de 0.07441 > 0.05, superior al umbal, por otro lado El grupo 2 que estimaba la retencion de un 7 dias ha presentado un resultado de 0.00155 < 0.05. Los resultados nos indican que el grupos de retencion a los 7 dias, obtendra mayor retencion'